In [1]:
from datasets import load_dataset
import random
from transformers import pipeline
import torch
from sentence_transformers import SentenceTransformer, util
import pandas as pd
#from tqdm import tqdm
from tqdm.notebook import tqdm
import evaluate


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.5 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/lib/python3/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/lib/python3/dist-packages/traitlets/config/application.py", line 846, in launch_instance
    app.start()
  File "/usr/lib/python3/dist-packages/ipykernel/kernelapp.py", line 677, in start
    s

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.5 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/lib/python3/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/lib/python3/dist-packages/traitlets/config/application.py", line 846, in launch_instance
    app.start()
  File "/usr/lib/python3/dist-packages/ipykernel/kernelapp.py", line 677, in start
    s

AttributeError: _ARRAY_API not found

2025-04-30 17:49:10.620137: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746053350.664767  263677 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746053350.678068  263677 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746053350.755544  263677 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746053350.755566  263677 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746053350.755570  263677 computation_placer.cc:177] computation placer alr

In [2]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

# Loading Dataset

The dataset can be found at https://huggingface.co/datasets/code-search-net/code_search_net

In [3]:
dataset = load_dataset('code-search-net/code_search_net')

Here are the columns in the dataset. We will be using `func_documentation_string` for documentation and `func_code_string` for code.

In [4]:
dataset['train'][0].keys()

dict_keys(['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'])

Getting a random sample from the test split to test our baseline model. We will increase the size once we finalize the experimental plan

In [ ]:
#dataset_sample = random.choices(dataset['test'], k=50)
dataset_sample = []
while len(dataset_sample) < 50:
    message_string = random.choices(dataset['test'], k=1)
    full_message = message_string[0]['func_documentation_string'] + message_string[0]['func_code_string']
    if len(full_message) / 4 < 4000:
        dataset_sample.append(message_string)

# Testing the Baseline model

Here we define the system prompt for the Llama 2 model.

In [ ]:
prompt_llama = \
'''You are a helpful agent designed to simplify code documentation for beginner programmers.
You will be provided with a block of code and the existing doucmentation that accompanies it.
Simplify the given documentation, using the provided code as context, so that it is understandable
to beginner programmers. Output absolutely nothing else besides the simplified documentation.
Make sure to keep any documentation formatting codes present in the simplified documentation.
If you feel that the existing documentation is simple enough and meaning would be lost by simplifying
it further, feel free to keep the documentation as is. Here is the original documentation and code:'''

Creating the pipeline for the Llama 2 model using the HuggingFace transformers library. Modified from the example here: https://huggingface.co/docs/transformers/en/model_doc/llama2

In [ ]:
pipe_llama = pipeline(
    task="text-generation",
    model="meta-llama/Llama-2-7b-chat-hf",
    torch_dtype=torch.float16,
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
)

OSError: We couldn't connect to 'https://huggingface.co' to load the files, and couldn't find them in the cached files.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

Testing the pipeline. Modified from examples given here: https://huggingface.co/docs/transformers/main/en/chat_templating

In [ ]:
message = [
    {"role": "system", "content": prompt_llama},
    {"role": "user", "content": f"Documentation:\n{dataset['test'][0]['func_documentation_string']}\n\nCode:\n{dataset['test'][0]['func_code_string']}"}
]
print(f"Original Documentation:\n{dataset['test'][0]['func_documentation_string']}\n")
print(f"Code:\n{dataset['test'][0]['func_code_string']}\n")
print(pipe_llama(message, pad_token_id=pipe_llama.tokenizer.eos_token_id)[0]['generated_text'][-1]['content'])

Original Documentation:
Extracts video ID from URL.

Code:
def get_vid_from_url(url):
        """Extracts video ID from URL.
        """
        return match1(url, r'youtu\.be/([^?/]+)') or \
          match1(url, r'youtube\.com/embed/([^/?]+)') or \
          match1(url, r'youtube\.com/v/([^/?]+)') or \
          match1(url, r'youtube\.com/watch/([^/?]+)') or \
          parse_query_param(url, 'v') or \
          parse_query_param(parse_query_param(url, 'u'), 'v')



NameError: name 'pipe' is not defined

Loading the evaluation model used for computing semantic similarity. Taken from example here: https://huggingface.co/tasks/sentence-similarity

In [ ]:
eval_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

Running inference on the dataset sample

In [ ]:
semantic_similarities = []
metrics = evaluate.combine(['rouge', 'meteor'])

for instance in tqdm(dataset_sample):
    message = [
        {"role": "system", "content": prompt_llama},
        {"role": "user", "content": f"Documentation:\n{instance[0]['func_documentation_string']}\n\nCode:\n{instance[0]['func_code_string']}"}
    ]

    result = pipe_llama(message, pad_token_id=pipe_llama.tokenizer.eos_token_id)[0]['generated_text'][-1]['content']

    embedding_original = eval_model.encode(instance[0]['func_documentation_string'], convert_to_tensor=True)
    embedding_predicted = eval_model.encode(result, convert_to_tensor=True)

    semantic_similarities.append(util.pytorch_cos_sim(embedding_original, embedding_predicted).item())
    metrics.add(predictions=result, references=instance[0]['func_documentation_string'])

Summary statistics for semantic similarity results

In [ ]:
pd.DataFrame(semantic_similarities).describe()

ROUGE and METEOR results

In [ ]:
metrics.compute()

# Testing the Alternative model

Here we define the system prompt for the Gemma 3 model.

In [ ]:
pipe_gemma = pipeline(
    "text2text-generation",
    model="google/gemma-3-4b-it",
    torch_dtype=torch.float32,
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
)
"""pipe_gemma = pipeline(
    "text2text-generation",
    model="google/gemma-3-4b-it",
    torch_dtype=torch.float32,
    device=torch.device("cpu")
)"""

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cpu
The model 'Gemma3ForConditionalGeneration' is not supported for text2text-generation. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'Qwen2AudioForConditionalGeneration', 'SeamlessM4TForTextToText', 'SeamlessM4Tv2ForTextToText', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'UMT5ForConditionalGeneration', 'XLMProphetNetForConditio

In [12]:
comment = dataset['test'][0]['func_documentation_string']
code = dataset['test'][0]['func_code_string']

In [ ]:
def llm_prompt(comment, code):
    prompt_gemma = \
    '''You are a helpful agent designed to simplify code documentation for beginner programmers.
    You will be provided with a block of code and the existing doucmentation that accompanies it.
    Using the provided code as context, give a simplified explanation of the code so that it is understandable
    to beginner programmers. Output absolutely nothing else besides the simplified explanation.
    Make sure to keep any documentation formatting codes present in the simplified explanation.
    If you feel that the existing documentation is simple enough and meaning would be lost by simplifying
    it further, feel free to keep the documentation as is. Here is the original documentation and code: \nDocumenation:\n{Documentation}\n\nCode:\n{CODE}'''.format(Documentation=comment, CODE=code)
    
    return prompt_gemma

In [37]:
#print(pipe_gemma(message))
message = "Give me a number between 1 and 10"
output = pipe_gemma(message, max_new_tokens=20)
text = output[0]['generated_text']
result = text.replace(message, "")
print(result)

.
Okay, I'll give you 7.
```python
def find_largest


In [16]:
dataset_sample = random.choices(dataset['test'], k=50)

In [ ]:
gemma_semantic_similarities = []
gemma_metrics = evaluate.combine(['rouge', 'meteor'])

for instance in tqdm(dataset_sample):
    """message = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": f"Documentation:\n{instance['func_documentation_string']}\n\nCode:\n{instance['func_code_string']}"}
    ]"""
    new_prompt = llm_prompt(instance['func_documentation_string'], instance['func_code_string'])

    text = pipe_gemma(new_prompt, max_new_tokens=30)[0]['generated_text']
    result = text.replace(new_prompt, "")
    

    embedding_original = eval_model.encode(instance['func_documentation_string'], convert_to_tensor=True)
    embedding_predicted = eval_model.encode(result, convert_to_tensor=True)

    gemma_semantic_similarities.append(util.pytorch_cos_sim(embedding_original, embedding_predicted).item())
    gemma_metrics.add(predictions=result, references=instance['func_documentation_string'])

[nltk_data] Downloading package wordnet to /home/adeniji/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/adeniji/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/adeniji/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


  0%|          | 0/50 [00:00<?, ?it/s]

In [41]:
pd.DataFrame(gemma_semantic_similarities).describe()

,0
count,50.000000
mean,0.072351
std,0.064176
min,-0.091277
25%,0.025552
50%,0.071694
75%,0.119409
max,0.229634


In [42]:
gemma_metrics.compute()

{'rouge1': np.float64(0.06742711817447161),
 'rouge2': np.float64(0.0030681818181818176),
 'rougeL': np.float64(0.0578422116177936),
 'rougeLsum': np.float64(0.058901884008119756),
 'meteor': np.float64(0.061475878441703616)}